# build toy model and save it to pickle

In [2]:
import pickle

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Train the model

iris = load_iris()
data, labels = iris.data, iris.target
training_data, test_data, training_labels, test_labels = train_test_split(data, labels)

model = LogisticRegression(solver="lbfgs", multi_class="multinomial")
model.fit(training_data, training_labels)
accuracy = model.score(test_data, test_labels)
print("accuracy: {:.2f}".format(accuracy))

# Upload the model

pickle.dump(model, open("toy_model.pkl", "wb"))

accuracy: 0.97


/Users/kai/anaconda3/envs/dlnn/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


## Write Predictor Class

In [36]:
%%writefile predictor.py

import os
import pickle

labels = ["setosa", "versicolor", "virginica"]


class PythonPredictor:
    def __init__(self, config):
        """
        if os.environ.get("AWS_ACCESS_KEY_ID"):
            s3 = boto3.client("s3")  # client will use your credentials if available
        else:
            s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))  # anonymous client

        s3.download_file(config["bucket"], config["key"], "/tmp/model.pkl")
        """
        self.model = pickle.load(open("toy_model.pkl", "rb"))

    def predict(self, payload):
        measurements = [
            payload["sepal_length"],
            payload["sepal_width"],
            payload["petal_length"],
            payload["petal_width"],
        ]

        label_id = self.model.predict([measurements])[0]
        return labels[label_id]

Overwriting predictor.py


## Deployment Configuration

In [37]:
%%writefile cortex.yaml
- name: iris-model
  kind: RealtimeAPI
  predictor:
    type: python
    path: predictor.py
    config:
      bucket: foo-bucket
      key: bar-folder/model-foobar.pkl

Overwriting cortex.yaml


## Deployment Requirements

In [38]:
%%writefile requirements.txt
pandas
scikit-learn

Overwriting requirements.txt


## Sample Query Payload

In [11]:
%%writefile sample.json
{
  "sepal_length": 5.2,
  "sepal_width": 3.6,
  "petal_length": 1.5,
  "petal_width": 0.3
}

Writing sample.json


## Deploy Model
deploy the model in terminal

In [12]:
# !cortex deploy
# !cortex get 
# !cortex get recommender

## Query Deployed Model

In [44]:
!curl http://localhost:8888 -X POST -H "Content-Type: application/json"

<html><title>403: Forbidden</title><body>403: Forbidden</body></html>